In [1]:
import gensim.summarization.bm25 
import gensim
import os
import numpy as np
import pickle
import spacy
import torch
import transformers
import pickle
import numpy as np
from nltk import SnowballStemmer
import re

In [27]:
import bs4
import urllib.request
import re
import timeit
import os
import spacy

class WebsiteData():
    
    def __init__(self, website,depth, pathName):#,website,parser = 'lxml'):
        '''
        Constructor of the class. We let the user choose the parser type, but keep it default to the fastest parser.
        We are keeping the bsObject private because the end user should NOT get access to any HTML information.
        '''
        self.website = website
        self.pageDict = {}
        self.depth = depth
        self.pathName = pathName
        '''
        try:
            sauce = urllib.request.urlopen(self.website)
            self.__bsObject = bs4.BeautifulSoup(sauce,parser)
        except:
            print("Website not accessible : " ,self.website)
            self.__bsObject = None
            return None
        '''
            
    def parseBaseWebsite(self, parser = 'lxml'):
        try:
            sauce = urllib.request.urlopen(self.website)
            self.__bsObject = bs4.BeautifulSoup(sauce,parser)
            return 1
        except Exception as e:
            print("Website not accessible. Exception :  " ,e)
            self.__bsObject = None
            return None
    
    def __getBaseURL(self):
        '''
        Return the base url of the website being parsed
        '''
        count = 0
        base1 = []
        base = ''
        if re.findall('http[s]?', self.website):
            for i in self.website:
                if i == '/':
                    count= count+1
                if count == 3:
                    break
                base1.append(i)
        
            base = base.join(base1)
        
        return base
    
    def displayHtml(self):
        '''
        Display website in HTML Structure 
        '''
        print(self.__bsObject.prettify())
    
    def retrieveHrefs(self):
        '''
        Retrieve links from the HREF Tags in the webpage 
        '''
        self.__linksList = []
        baseUrl = self.__getBaseURL()
        print("base url : ", baseUrl)
        
       
        bodyContent = self.__bsObject.find("div", attrs = {'id': 'mntl-sc-page_1-0'})
        
        
        for link in bodyContent.find_all('a'):
            #print(link)
            suffixURL = link.get('href')
            #print(type(suffixURL))
            print(suffixURL)
            
            if (suffixURL is not None) and ('www.investopedia.com' in suffixURL) and (suffixURL.endswith('.asp'))  :
                self.__linksList.append(str(suffixURL))
                
         
        return self.__linksList
    
    def saveWebsiteData(self, levels):
        '''
        Save the data from the website in a txt file. The method creates a file with the topic of the link as the file name.
        The code will be extended to store the data from all the links in the linksList in the future.
        '''
        pathName = self.pathName
        if levels == 0 :
            pathName = pathName
        else:
            for counter in range(levels):
                pathName = pathName +  "/level" + str(counter+1 )
                
        if not os.path.exists(os.path.join(os.getcwd(),pathName)):
            os.mkdir(os.path.join(os.getcwd(),pathName ))
                

        #headerContent = self.__bsObject.find_all("span", attrs = {'class': 'mntl-sc-block-heading__text'})
 
            
            
        
        #print(headerContent)
        bodyContent = self.__bsObject.find("div", attrs = {'id': 'mntl-sc-page_1-0'})
        paragraphs = bodyContent.find_all('p')
        #headers = headerContent.find_all('')

        print("writing data")        
        file_name = os.path.join(os.path.join(os.getcwd(), pathName), (self.website).split("/")[-1] + ".txt")
        print(file_name)
        #with open(file_name ,"w+", encoding='utf-8') as f:
        maxParaLen = 0
        sumParaLen = 0
        nlp = spacy.load("en_core_web_sm",disable=['ner', 'parser', 'textcat'])
        totalParaLen = len(paragraphs)
        #print(paragraphs)
        count = 0 
        prevHeadingText = ""
        fileText = ""
        nlpModel = spacy.load("en_core_web_sm")
        
        for paragraph in paragraphs:
            fileName = (self.website).split("/")[-1]
            fileName = fileName.split(".asp")[0]
            #print(fileName)
            #with open( os.path.join(os.path.join(os.getcwd(), pathName) , fileName + str(count)+ ".txt"),"w+", encoding='utf-8') as f:
               
            
            if paragraph.find_parent('div').find_previous('h3') is not None:
                heading = paragraph.find_parent('div').find_previous('h3')
                if heading.find('span') is not None:
                    heading_text = heading.find('span').text
                    
                else:
                    heading_text = heading.text
            else:
                heading = paragraph.find_parent('div').find_previous('h1')
                heading_text = heading.text
            
            
            heading_text = heading_text.replace('\n', "")
            heading_clean = re.sub('[^A-Za-z0-9]+', '', heading_text)
            
            
            
            if heading_text != prevHeadingText:
                sizeBuffer = 0
                counter = 0
                paraText = heading_text+ ". " + paragraph.getText()
                fileText = paraText
                fileTextArray = nlp(fileText)
                prevHeadingText = heading_text
                file_clean_name = heading_clean
                sizeBuffer = len([tokens.text for tokens in fileTextArray])
                print(" sizeBuffer1 : ", sizeBuffer )
                with open( os.path.join(os.path.join(os.getcwd(), pathName) , file_clean_name + ".txt"),"w+", encoding='utf-8') as f:
                    f.write(fileText)
                    
                
            else:
                #print(paragraph)
                paragraphDoc = nlp(paragraph.getText())
                paragraphText = [tokens.text for tokens in paragraphDoc]
                print("this condition :" , len(paragraphText) + sizeBuffer)
                if (len(paragraphText) + sizeBuffer) > 460:
                    
                    print ("special case :", heading_clean + str(counter+1) )
                    counter = counter + 1
                    heading_clean =  heading_clean + str(counter)
                    file_clean_name = heading_clean
                    fileText = heading_text+ ". " + paragraph.getText()
                    sizeBuffer = len([tokens.text for tokens in fileTextArray])
                    print(" sizeBuffer2 : ", sizeBuffer )
                    with open( os.path.join(os.path.join(os.getcwd(), pathName) , file_clean_name + ".txt"),"w+", encoding='utf-8') as f1:
                        f1.write(fileText)
                        
                    
                else:
                    #fileText = fileText + " " + paragraph.getText()
                    fileText = " " + paragraph.getText()
                    sizeBuffer = sizeBuffer + len([tokens.text for tokens in fileTextArray])
                    print(" sizeBuffer3 : ", sizeBuffer )
                    with open( os.path.join(os.path.join(os.getcwd(), pathName) , file_clean_name + ".txt"),"a", encoding='utf-8') as f2:
                        f2.write(fileText)
                        
                
           
            
            '''
            paraText = paragraph.getText()
            print("paraText   :   ", paraText)
            paraTextDoc = nlp(paraText)
            paraTextDocList = [tokens.text for tokens in paraTextDoc if tokens.text.strip() != '']
            paraLen = len(paraTextDocList)
            paraText = " ".join(paraTextDocList)
            sumParaLen = sumParaLen + paraLen
            if paraLen > maxParaLen:
                maxParaLen = paraLen

            f.write(paraText)
            '''
            
            count+=1
                

            #avgParaLen = (sumParaLen/totalParaLen)
            #self.pageDict[(self.website).split("/")[-1]] = (maxParaLen , avgParaLen)

            #f.close()
        return self.pageDict
        


class WebsiteParser:
    
    def __init__(self,baseWebsite,level, pathName):
        
        self.baseWebsite = baseWebsite
        self.level = level
        self.pathName = pathName
    
    def parseWebsite(self):
        
        pageDict ={}
        linkDict = {}
        
        for level in range(self.level):
            
            if level == 0:
                obj = WebsiteData(self.baseWebsite,level, self.pathName)
                objret = obj.parseBaseWebsite()
                linkDict[level +1] = obj.retrieveHrefs()
                if objret is not None: 
                    data = obj.saveWebsiteData(level) 
                    pageDict.update(data)
            else:
                linkDict[level +1] = []
        
                if level < (self.level - 1):
                
                    for link in set(linkDict[level]):
                        obj1 = WebsiteData(link, level, self.pathName)
                        obj1ret = obj1.parseBaseWebsite()
                       
                        linkDict[level +1].extend(obj1.retrieveHrefs())
                        
                        if obj1ret is not None: 
                            data = obj1.saveWebsiteData(level) 
                            pageDict.update(data)
                
                else:
                    for link in set(linkDict[level]):
                        obj1 = WebsiteData(link, level, self.pathName)
                        obj1ret = obj1.parseBaseWebsite()
                        
                        if obj1ret is not None: 
                            data = obj1.saveWebsiteData(level) 
                            pageDict.update(data)

In [28]:
numLevels = 3
initLink = 'https://www.investopedia.com/terms/r/riskmanagement.asp'
#initLink = "https://www.investopedia.com/terms/e/equity.asp"
path = r"C:\Users\piyush.kathuria\Documents\Neural_IR\final_app\invest-data"
obj = WebsiteParser(initLink,numLevels,path)
print(obj.parseWebsite())

base url :  https://www.investopedia.com
https://www.investopedia.com/terms/f/financialplanner.asp
https://www.investopedia.com/terms/r/risk-analysis.asp
https://www.investopedia.com/ask/answers/042415/what-difference-between-moral-hazard-and-adverse-selection.asp
https://www.investopedia.com/articles/pf/07/risk_tolerance.asp
None
https://www.investopedia.com/terms/u/ustreasury.asp
https://www.investopedia.com/terms/d/derivative.asp
https://www.investopedia.com/terms/o/option.asp
https://www.investopedia.com/terms/f/futures.asp
https://www.investopedia.com/terms/s/subprime-meltdown.asp
https://www.investopedia.com/terms/m/mbs.asp
None
https://www.investopedia.com/terms/b/benchmark.asp
https://www.investopedia.com/terms/v/volatility.asp
None
https://www.investopedia.com/terms/s/standarddeviation.asp
https://www.investopedia.com/terms/d/dispersion.asp
https://www.investopedia.com/terms/a/annualized-total-return.asp
https://www.investopedia.com/terms/s/sp500.asp
None
https://www.investope

this condition : 157
 sizeBuffer3 :  102
 sizeBuffer1 :  85
this condition : 181
 sizeBuffer3 :  170
base url :  https://www.investopedia.com
None
https://www.investopedia.com/terms/r/riskmanagement.asp
https://www.investopedia.com/terms/m/moralhazard.asp
https://www.investopedia.com/terms/a/asymmetricinformation.asp
https://www.investopedia.com/terms/a/asymmetricinformation.asp
None
https://www.investopedia.com/articles/pf/06/insurancecontracts.asp
None
https://www.investopedia.com/terms/a/adverseselection.asp
None
https://www.investopedia.com/ask/answers/040815/what-are-some-examples-moral-hazard-business-world.asp
https://www.investopedia.com/ask/answers/032615/what-difference-between-moral-hazard-and-morale-hazard.asp
None
https://www.investopedia.com/terms/i/insurance-premium.asp
https://www.investopedia.com/articles/insurance/082516/examples-adverse-selection-insurance-industry.asp
https://www.investopedia.com/terms/l/lifeexpectancy.asp
writing data
C:\Users\piyush.kathuria\Docum

 sizeBuffer1 :  60
this condition : 91
 sizeBuffer3 :  120
this condition : 136
 sizeBuffer3 :  180
this condition : 249
 sizeBuffer3 :  240
this condition : 255
 sizeBuffer3 :  300
 sizeBuffer1 :  46
this condition : 121
 sizeBuffer3 :  92
this condition : 193
 sizeBuffer3 :  138
this condition : 176
 sizeBuffer3 :  184
this condition : 241
 sizeBuffer3 :  230
 sizeBuffer1 :  47
this condition : 135
 sizeBuffer3 :  94
this condition : 247
 sizeBuffer3 :  141
this condition : 229
 sizeBuffer3 :  188
this condition : 277
 sizeBuffer3 :  235
this condition : 329
 sizeBuffer3 :  282
this condition : 354
 sizeBuffer3 :  329
this condition : 402
 sizeBuffer3 :  376
this condition : 465
special case : CommonFormsofDerivatives1
 sizeBuffer2 :  47
this condition : 97
 sizeBuffer3 :  94
this condition : 154
 sizeBuffer3 :  141
this condition : 229
 sizeBuffer3 :  188
this condition : 274
 sizeBuffer3 :  235
this condition : 311
 sizeBuffer3 :  282
this condition : 378
 sizeBuffer3 :  329
this c

 sizeBuffer1 :  70
 sizeBuffer1 :  102
this condition : 167
 sizeBuffer3 :  204
 sizeBuffer1 :  13
 sizeBuffer1 :  27
this condition : 172
 sizeBuffer3 :  54
this condition : 103
 sizeBuffer3 :  81
this condition : 188
 sizeBuffer3 :  108
this condition : 197
 sizeBuffer3 :  135
this condition : 195
 sizeBuffer3 :  162
 sizeBuffer1 :  76
this condition : 137
 sizeBuffer3 :  152
this condition : 244
 sizeBuffer3 :  228
base url :  https://www.investopedia.com
None
None
https://www.investopedia.com/terms/r/regression.asp
https://www.investopedia.com/terms/s/sp500.asp
https://www.investopedia.com/terms/l/least-squares-method.asp
None
writing data
C:\Users\piyush.kathuria\Documents\Neural_IR\final_app\invest-data/level1\line-of-best-fit.asp.txt
 sizeBuffer1 :  92
 sizeBuffer1 :  65
this condition : 153
 sizeBuffer3 :  130
this condition : 230
 sizeBuffer3 :  195
this condition : 290
 sizeBuffer3 :  260
 sizeBuffer1 :  32
this condition : 40
 sizeBuffer3 :  64
this condition : 247
 sizeBuff

 sizeBuffer1 :  64
 sizeBuffer1 :  59
this condition : 284
 sizeBuffer3 :  118
 sizeBuffer1 :  57
this condition : 143
 sizeBuffer3 :  114
this condition : 181
 sizeBuffer3 :  171
 sizeBuffer1 :  66
this condition : 158
 sizeBuffer3 :  132
 sizeBuffer1 :  83
this condition : 138
 sizeBuffer3 :  166
 sizeBuffer1 :  86
this condition : 139
 sizeBuffer3 :  172
this condition : 255
 sizeBuffer3 :  258
this condition : 326
 sizeBuffer3 :  344
 sizeBuffer1 :  60
this condition : 139
 sizeBuffer3 :  120
this condition : 183
 sizeBuffer3 :  180
 sizeBuffer1 :  74
this condition : 137
 sizeBuffer3 :  148
base url :  https://www.investopedia.com
https://www.investopedia.com/terms/s/sector.asp
https://www.investopedia.com/terms/m/market_cycles.asp
https://www.investopedia.com/terms/h/holdings.asp
https://www.investopedia.com/terms/h/holdingperiod.asp
None
https://www.investopedia.com/terms/p/portfoliomanager.asp
https://www.investopedia.com/terms/n/nber.asp
None
None
https://fundresearch.fidelity

...

 sizeBuffer1 :  126
this condition : 204
 sizeBuffer3 :  252
this condition : 278
 sizeBuffer3 :  378
 sizeBuffer1 :  122
this condition : 189
 sizeBuffer3 :  244
this condition : 285
 sizeBuffer3 :  366
this condition : 416
 sizeBuffer3 :  488
this condition : 555
special case : ChecktheFundamentals1
 sizeBuffer2 :  122
 sizeBuffer1 :  136
this condition : 166
 sizeBuffer3 :  272
this condition : 324
 sizeBuffer3 :  408
 sizeBuffer1 :  45
this condition : 97
 sizeBuffer3 :  90
this condition : 149
 sizeBuffer3 :  135
this condition : 177
 sizeBuffer3 :  180
 sizeBuffer1 :  78
this condition : 199
 sizeBuffer3 :  156
 sizeBuffer1 :  97
writing data
C:\Users\piyush.kathuria\Documents\Neural_IR\final_app\invest-data/level1/level2\product_recall.asp.txt
 sizeBuffer1 :  52
 sizeBuffer1 :  104
this condition : 147
 sizeBuffer3 :  208
this condition : 268
 sizeBuffer3 :  312
 sizeBuffer1 :  105
this condition : 186
 sizeBuffer3 :  210
this condition : 266
 sizeBuffer3 :  315
writing data
C:\

writing data
C:\Users\piyush.kathuria\Documents\Neural_IR\final_app\invest-data/level1/level2\bearmarket.asp.txt
 sizeBuffer1 :  90
this condition : 144
 sizeBuffer3 :  180
 sizeBuffer1 :  74
this condition : 90
 sizeBuffer3 :  148
 sizeBuffer1 :  39
this condition : 103
 sizeBuffer3 :  78
 sizeBuffer1 :  81
this condition : 145
 sizeBuffer3 :  162
 sizeBuffer1 :  15
 sizeBuffer1 :  102
this condition : 174
 sizeBuffer3 :  204
 sizeBuffer1 :  104
this condition : 170
 sizeBuffer3 :  208
 sizeBuffer1 :  77
this condition : 154
 sizeBuffer3 :  154
 sizeBuffer1 :  66
this condition : 110
 sizeBuffer3 :  132
writing data
C:\Users\piyush.kathuria\Documents\Neural_IR\final_app\invest-data/level1/level2\real-property.asp.txt
 sizeBuffer1 :  87
 sizeBuffer1 :  55
this condition : 116
 sizeBuffer3 :  110
 sizeBuffer1 :  61
 sizeBuffer1 :  63
 sizeBuffer1 :  89
 sizeBuffer1 :  110
this condition : 208
 sizeBuffer3 :  220
writing data
C:\Users\piyush.kathuria\Documents\Neural_IR\final_app\invest-

writing data
C:\Users\piyush.kathuria\Documents\Neural_IR\final_app\invest-data/level1/level2\insurance-premium.asp.txt
 sizeBuffer1 :  45
this condition : 93
 sizeBuffer3 :  90
 sizeBuffer1 :  89
 sizeBuffer1 :  22
this condition : 37
 sizeBuffer3 :  44
this condition : 156
 sizeBuffer3 :  66
this condition : 112
 sizeBuffer3 :  88
 sizeBuffer1 :  53
this condition : 149
 sizeBuffer3 :  106
this condition : 173
 sizeBuffer3 :  159
this condition : 252
 sizeBuffer3 :  212
this condition : 299
 sizeBuffer3 :  265
writing data
C:\Users\piyush.kathuria\Documents\Neural_IR\final_app\invest-data/level1/level2\exchange-traded-derivative.asp.txt
 sizeBuffer1 :  129
 sizeBuffer1 :  67
this condition : 140
 sizeBuffer3 :  134
 sizeBuffer1 :  126
writing data
C:\Users\piyush.kathuria\Documents\Neural_IR\final_app\invest-data/level1/level2\tranches.asp.txt
 sizeBuffer1 :  74
 sizeBuffer1 :  89
this condition : 137
 sizeBuffer3 :  178
this condition : 202
 sizeBuffer3 :  267
 sizeBuffer1 :  134
th

writing data
C:\Users\piyush.kathuria\Documents\Neural_IR\final_app\invest-data/level1/level2\scenario_analysis.asp.txt
 sizeBuffer1 :  85
this condition : 105
 sizeBuffer3 :  170
 sizeBuffer1 :  73
this condition : 130
 sizeBuffer3 :  146
this condition : 245
 sizeBuffer3 :  219
 sizeBuffer1 :  90
this condition : 155
 sizeBuffer3 :  180
this condition : 267
 sizeBuffer3 :  270
this condition : 332
 sizeBuffer3 :  360
writing data
C:\Users\piyush.kathuria\Documents\Neural_IR\final_app\invest-data/level1/level2\vix.asp.txt
 sizeBuffer1 :  125
 sizeBuffer1 :  124
this condition : 128
 sizeBuffer3 :  248
this condition : 306
 sizeBuffer3 :  372
this condition : 415
 sizeBuffer3 :  496
this condition : 669
special case : HowDoestheVIXWork1
 sizeBuffer2 :  124
this condition : 346
 sizeBuffer3 :  248
this condition : 285
 sizeBuffer3 :  372
this condition : 546
special case : HowDoestheVIXWork2
 sizeBuffer2 :  124
this condition : 152
 sizeBuffer3 :  248
this condition : 404
 sizeBuffer3 :

writing data
C:\Users\piyush.kathuria\Documents\Neural_IR\final_app\invest-data/level1/level2\accepting-risk.asp.txt
 sizeBuffer1 :  117
 sizeBuffer1 :  125
this condition : 221
 sizeBuffer3 :  250
 sizeBuffer1 :  30
writing data
C:\Users\piyush.kathuria\Documents\Neural_IR\final_app\invest-data/level1/level2\efficientmarkethypothesis.asp.txt
 sizeBuffer1 :  69
this condition : 102
 sizeBuffer3 :  138
this condition : 175
 sizeBuffer3 :  207
 sizeBuffer1 :  52
this condition : 151
 sizeBuffer3 :  104
 sizeBuffer1 :  39
this condition : 162
 sizeBuffer3 :  78
this condition : 135
 sizeBuffer3 :  117
writing data
C:\Users\piyush.kathuria\Documents\Neural_IR\final_app\invest-data/level1/level2\optionscontract.asp.txt
 sizeBuffer1 :  44
this condition : 93
 sizeBuffer3 :  88
 sizeBuffer1 :  68
this condition : 127
 sizeBuffer3 :  136
this condition : 193
 sizeBuffer3 :  204
 sizeBuffer1 :  60
this condition : 130
 sizeBuffer3 :  120
 sizeBuffer1 :  75
 sizeBuffer1 :  66
this condition : 12

In [2]:
class DocumentRanker():
    
    def __init__(self, query, folderPath):
        
        self.documentFolderPath = folderPath
        print("folderPath  : ", folderPath)
        self.query = self.convertQuery(query)
        self.folderDocumentContent = self.loadDocuments()  # Store the content of all the files in dictionary
        self.bm25Model = self.createBM25Model()
    
    
    def convertQuery(self,query):
        
        nlp = spacy.load("en_core_web_sm",disable = ['ner', 'parser', 'textcat'])
        doc = nlp(query.lower())
        return doc
        
    def loadDocuments(self):
        '''
        Load documents from the invest_data folder into a dictionary
        '''
        documentTuple = []
        #nounList = self.getNouns()
        nlp = spacy.load("en_core_web_sm",disable = ['ner', 'parser', 'textcat'])
        
        for root, dirs, files in os.walk(self.documentFolderPath):
    
            for file in files:
                if file.endswith('txt'):
                    with open(os.path.abspath(os.path.join(root, file)), 'r' , encoding="utf8" ) as f:
                        content = f.read()
                        content = content.lower()
                        doc = nlp(content)
                        tokenizedContent = [tokens.text for tokens in doc if tokens.text]         # Change 1 
                        documentTuple.append((file,tokenizedContent))
        
        
        
        return documentTuple
    
        
    
    def getNouns(self):
        
        #nlp = spacy.load("en_core_web_sm")
        
        '''
        spacy_stopwords = list(spacy.lang.en.stop_words.STOP_WORDS)
        
        querySplit = self.query.lower().split()
        
        querySplit = [x for x in querySplit if x not in spacy_stopwords]
        
        print(querySplit)
        
        return querySplit
        
        '''
        nounList = []
        for token in self.query:
            #print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_, token.shape_, token.is_alpha, token.is_stop)
            if token.pos_ == "NOUN" or token.pos_ == "PROPN" or token.pos_ == "ADJ":
                print(token.text)
                print(token.pos_)
                nounList.append(token.text)
            
        print("noun list : ", nounList)        
        return nounList
    
        
    def createBM25Model(self):
        '''
        Return a created BM25 model.
        '''
        return gensim.summarization.bm25.BM25([x[1] for x in self.folderDocumentContent])
    
    def rankDocuments(self):
        '''
        Rank the documents in the corpus wrt the query. 
        '''
        query = [tokens.text for tokens in self.query if not tokens.is_punct]
        scores = self.bm25Model.get_scores(query)
        documentScores =  [(self.folderDocumentContent[i][0],scores[i]) for i in range(len(scores))]
        return documentScores
    
    def returnTopK(self,k=5):
        '''
        Return top k ranked documents wrt given user query
        '''
        
        scoreDict = self.rankDocuments()
        return sorted(scoreDict , key = lambda x: x[1], reverse = True)[:k]
    
    def returnTopDocuments(self):
        '''
        Return top documents wrt given user query. This uses a different approach from our 
        previous function. We return all documents with score a standard deviation above the
        mean score of our corpus
        '''
        
        scoreTup = self.rankDocuments()
        #scoreDict = dict(scoreTup)
        scores = np.array([x[1] for x in scoreTup])
        #scores = np.array(list(scoreDict.values()))
        meanScore = np.mean(scores)
        sdScore = np.std(scores)
        maxThreshold = meanScore + sdScore
        topScoresDict = [x for x in scoreTup if x[1] > maxThreshold]
        topDocumentScores = sorted(topScoresDict, key = lambda x: x[1], reverse = True)
        
        return topDocumentScores
    
    def returnTopDocumentsData(self):
        '''
        Return data from the top documents retrieved by returnTopDocuments
        '''
        #query = self.query
        #topDocumentScores = self.returnTopDocuments()
        topDocumentScores = self.returnTopK(20)
        print("topDocumentScores  : ", topDocumentScores)
        topDocumentNames = [x[0] for x in topDocumentScores]
        #print(topDocumentNames)
        
        topDocumentText = [x for x in self.folderDocumentContent if x[0] in topDocumentNames]
        return topDocumentText

In [29]:
class PassageRanker():
    
    def __init__(self, query, topDocumentContent):
        
        self.query = self.convertQuery(query)
        self.topDocumentContent = topDocumentContent
        #self.BM25Model = self.createBM25Model()
    
    def convertQuery(self,query):
        
        nlp = spacy.load("en_core_web_sm",disable = ['ner', 'parser', 'textcat'])
        doc = nlp(query.lower())
        return doc
    
    
    def createBM25Model(self,documentContent):
        '''
        Return a created BM25 model.
        '''
        #return gensim.summarization.bm25.BM25([x[1] for x in self.topDocumentContent])
        return gensim.summarization.bm25.BM25(documentContent)
    
    
    def getNouns(self):
        
        #nlp = spacy.load("en_core_web_sm")
        
        '''
        spacy_stopwords = list(spacy.lang.en.stop_words.STOP_WORDS)
        
        querySplit = self.query.lower().split()
        
        querySplit = [x for x in querySplit if x not in spacy_stopwords]
        
        print(querySplit)
        
        return querySplit
        
        '''
        nounList = []
        for token in self.query:
            #print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_, token.shape_, token.is_alpha, token.is_stop)
            if token.pos_ == "NOUN" or token.pos_ == "PROPN" or token.pos_ == "ADJ":
                nounList.append(token.text)
            
        #print("noun list : ", nounList)        
        return nounList
    
    def checkNounMatch(self,paragraph,nounList):
        '''
        A function to see if each noun is matching for a paragraph
        '''
        nounBool = True
        
        #nlp = spacy.load("en_core_web_sm",disable=['ner', 'parser', 'textcat'])
        stemmer = SnowballStemmer("english")
        paragraphContent = " ".join([stemmer.stem(token) for token in paragraph])
        
        
        for nouns in nounList:
            
            #paragraphDoc = nlp(paragraphContent)
            #paragraphLemma = [tokens.lemma_ for tokens in paragraphDoc]
            
            if stemmer.stem(nouns) in paragraphContent:
                nounBool = True
            else:
                nounBool = False
                break
        
        return nounBool
                
            
    def returnParagraphList(self,content):
        '''
        Return pargaraph in the form of list of lists to be fed to the BM25 Model
        '''
        paragraphList = []
        paragraph = content #[sepCounter:i] # " ".join() <-
        nounList = self.getNouns()
        if self.checkNounMatch(paragraph,nounList):     
            paragraphList.append(paragraph)
        return paragraphList
        
        '''
        paragraphList = []
        sepCounter = 0
        nounList = self.getNouns()
        prevI = [0]
        tempPara = []
        megaPara = []
        totalParaLen = 0
        counter = 1
        
        for i,term in enumerate(content):
            
            if term == "--------------------------":
                #if self.checkNounMatch(paragraph,nounList):
                    if totalParaLen < 450:
                        counter += 1
                        paragraph = content[sepCounter:i] # " ".join() <-
                        tempPara.append(paragraph)
                        sepCounter = i+1
                        prevI = prevI[counter-1]
                        totalParaLen += len(paragraph)
                    else:
                        megaPara.extend(tempPara)
                        tempPara = []
                        sepCounter = 
                        
                        
                if self.checkNounMatch(paragraph,nounList):
                #if all(nouns in paragraph for nouns in nounList):
                    paragraphList.append(paragraph)
                
                sepCounter = i+1
        
        return paragraphList
    '''
        '''
        nlp = spacy.load("en_core_web_sm",disable = ['ner', 'parser', 'textcat'])
        content = [x for x in content if "\n" not in x]
        documentContent = " ".join(content)
        paragraphsList = documentContent.split("--------------------------")
        bigPassage = []
        headingList = []
        megaList = []
        megaDict = {}
        bigString = ""
        for idx, i in enumerate(paragraphsList):
            heading = i.split(" . ")[0]

            text = i.split(" . ")[1:]
            text = " ".join(text)

            if heading not in headingList:
                
                if megaList != []:
                    print(headingList)
                    megaDict[headingList[headingList.index(heading.strip()) -1]] = megaList
                bigString  = bigString + heading +  ". "
                headingList.append(heading)
            
            if len(bigString.split()) + len(text.split()) < 480:#and heading in headingList:
                bigString  = bigString + " " + text

            else:

                megaList.append(bigString)
                bigString = text
                print("Length of Text  ", len(bigString.split()))

        
        documentContent = []
        
        for documents in megaList:
            
            documentDoc = nlp(documents)
            documentContent.append([tokens.text for tokens in documentDoc if '\n' not in tokens.text and tokens.text.strip()!= ''])
            
        
        print(len(documentContent))
    '''
        
   
    
    
    def returnParagraphScores(self,bm25Model,query):
        '''
        Return the score of a paragraph given a model.
        '''
        return bm25Model.get_scores(query)
        
    
    def returnTopPassages(self, k=10):
        '''
        Rank each paragraph from the document and return the top 10 passages from the collection of documents
        '''
        query = [tokens.text for tokens in self.query if not tokens.is_punct]
        
        #documentParagraphScores = [(document[0],self.rankPassages(document[1])) for document in topDocumentContent]
        paragraphScoreTup = []
        paragraphLoL = []
        for documents in self.topDocumentContent:
            #print(documents)
            paragraphLoL.extend(self.returnParagraphList(documents[1]))
        
        
        print(len(paragraphLoL))
        #passageBM25Model = self.createBM25Model(paragraphLoL)
        
        #for documents in self.topDocumentContent:
        #paragraphScores = self.returnParagraphScores(passageBM25Model, query)
        for i,paragraph in enumerate(paragraphLoL):
                
            #paragraphScoreTup.append((paragraph,paragraphScores[i]))
            paragraphScoreTup.append((paragraph,1))
        
        print(" length of paragraphScoreTup : ", len(paragraphScoreTup))
        
        self.topParagraphScores = sorted(paragraphScoreTup, key = lambda x: x[1], reverse = True)
        
        return self.topParagraphScores#[:k]

In [30]:
class bertQAModel():
    
    def __init__(self):
        
        self.tokenizerModel,self.bertQAModel = self.__initializeModel()
    
    def __initializeModel(self):
        '''
        Initialize the Beret Tokenizer and the QA model. Note that this is currently compatible with
        transformers module, NOT pytorch/tensorflow
        '''
        with open("./models/bertTokenizer.pkl","rb") as f:
            tokenizerModel = pickle.load(f)
        
        with open("./models/bertQAModel.pkl","rb") as f:
            bertQAModel = pickle.load(f)
        
        return tokenizerModel, bertQAModel
    
    def stringProcess(self,answer):
        answerSplit = answer.split(" ##")
        return "".join(answerSplit)
    
    def predict(self,text,question):
        '''
        Predict the answer given a passage and a question.
        '''
        input_text = "[CLS] " + question + " [SEP] " + text + " [SEP]"
        print("INPUT_TEXT : ")
        print(input_text)
        input_ids = self.tokenizerModel.encode(input_text)
        #print("TOKENIZED TEXT : ")
        #print(input_ids)
        token_type_ids = [0 if i <= input_ids.index(102) else 1 for i in range(len(input_ids))]
        #print("TOKEN TYPE IDS : ")
        #print(token_type_ids)
        start_scores, end_scores = self.bertQAModel(torch.tensor([input_ids]), token_type_ids=torch.tensor([token_type_ids]))
        
        
        # Normalized scores
        #startScoresVec = ( (startScoresVec - np.mean(startScoresVec))/np.std(startScoresVec))
        #endScoresVec = ( (endScoresVec - np.mean(endScoresVec))/np.std(endScoresVec))
        
        # Normalized Scores
        #startScoresVec = scale(np.reshape(startScoresVec,newshape=(-1,1)))
        #endScoresVec = scale(np.reshape(endScoresVec,newshape=(-1,1)))
        
        # Normalized Scores
        #softmax = torch.nn.Softmax()
        #start_scores = softmax(start_scores)
        #end_scores = softmax(end_scores)
        #endScoresVec = scale(np.reshape(endScoresVec,newshape=(-1,1)))
        
        #startScoresVec = ( (startScoresVec - np.min(startScoresVec))/np.ptp(startScoresVec))
        #endScoresVec = ( (endScoresVec - np.min(endScoresVec))/np.ptp(endScoresVec))
        
        
        all_tokens = self.tokenizerModel.convert_ids_to_tokens(input_ids)
        answer = ' '.join(all_tokens[torch.argmax(start_scores) : torch.argmax(end_scores)+1])
        startScoreInd,endScoreInd = torch.argmax(start_scores),torch.argmax(end_scores)
        startScoresVec = start_scores.detach().numpy().flatten()
        endScoresVec = end_scores.detach().numpy().flatten()
        
        startScoreMax, endScoreMax = startScoresVec[startScoreInd] , endScoresVec[endScoreInd]
        #avgScore = float(np.absolute(startScoreMax) + np.absolute(endScoreMax))
        #avgScore = float(startScoreMax + endScoreMax)
        print("ANSWER RETRIEVED : ")
        print(self.stringProcess(answer))
        return (self.stringProcess(answer),startScoreMax)

In [31]:
# C:\Users\piyush.kathuria\Documents\Neural_IR\final_app\invest-data
# C:\Users\piyush.kathuria\Desktop\invest-data
path = r"C:\Users\piyush.kathuria\Documents\Neural_IR\final_app\invest-data"

In [136]:
query = "Give me a real world example of volatility."

In [137]:
docRanker = DocumentRanker(query,path)
topDocuments = docRanker.returnTopDocumentsData()

folderPath  :  C:\Users\piyush.kathuria\Documents\Neural_IR\final_app\invest-data
topDocumentScores  :  [('ChoosingtheRightFinancialPlanner.txt', 17.940017372418815), ('RealWorldExampleoftheVIX.txt', 16.425932377571495), ('RealWorldExampleofVolatility.txt', 16.394505795197823), ('RealWorldExampleofVolatility.txt', 16.394505795197823), ('RealWorldExampleofZomma.txt', 14.7516489372298), ('RealWorldExampleofaSeasonalAdjustment.txt', 13.665635056436159), ('HowDoestheVIXWork2.txt', 13.648914934212975), ('RealWorldExamplesofETFs.txt', 13.620388748699831), ('OtherMeasuresofVolatility.txt', 13.598979120210728), ('OtherMeasuresofVolatility.txt', 13.598979120210728), ('WhatIsAnnualizedTotalReturn.txt', 13.404395058894739), ('RealWorldExampleofaPIPE.txt', 13.31249292009943), ('RealWorldExampleofaMarginCall1.txt', 13.263632970614745), ('RealWorldExampleofaWarBond.txt', 13.219288219616413), ('UnderstandingZomma.txt', 13.137983219963841), ('RealWorldExamples.txt', 13.09430274881101), ('HowDoestheVIX

In [138]:
passageRanker = PassageRanker(query , topDocuments)
passageRanking = passageRanker.returnTopPassages()

5
 length of paragraphScoreTup :  5


In [139]:
passageRanking

[([' ',
   'real',
   'world',
   'example',
   'of',
   'volatility',
   '.',
   'suppose',
   'that',
   'an',
   'investor',
   'is',
   'building',
   'a',
   'retirement',
   'portfolio',
   '.',
   'since',
   'she',
   'is',
   'retiring',
   'within',
   'the',
   'next',
   'few',
   'years',
   ',',
   'she',
   "'s",
   'seeking',
   'stocks',
   'with',
   'low',
   'volatility',
   'and',
   'steady',
   'returns',
   '.',
   'she',
   'considers',
   'two',
   'companies',
   ':',
   'the',
   'investor',
   'would',
   'likely',
   'choose',
   'microsoft',
   'corporation',
   'for',
   'their',
   'portfolio',
   'since',
   'it',
   'has',
   'less',
   'volatility',
   'and',
   'more',
   'predictable',
   'short',
   '-',
   'term',
   'value',
   '.',
   '(',
   'for',
   'related',
   'reading',
   ',',
   'see',
   '"',
   'how',
   'to',
   'bet',
   'on',
   'volatility',
   'when',
   'the',
   'vxx',
   'expires',
   '"',
   ')'],
  1),
 ([' ',
   'how',
   

In [140]:
bertQA = bertQAModel()
answerTuple = []
nlp = spacy.load("en_core_web_sm")
queryDoc = nlp(query)
query = " ".join([tokens.text for tokens in queryDoc if not tokens.is_punct])

for passage,_ in passageRanking[:10] : 
            
    paraText = " ".join(passage)
    answerTuple.append(bertQA.predict(paraText, query))

INPUT_TEXT : 
[CLS] Give me a real world example of volatility [SEP]   real world example of volatility . suppose that an investor is building a retirement portfolio . since she is retiring within the next few years , she 's seeking stocks with low volatility and steady returns . she considers two companies : the investor would likely choose microsoft corporation for their portfolio since it has less volatility and more predictable short - term value . ( for related reading , see " how to bet on volatility when the vxx expires " ) [SEP]
ANSWER RETRIEVED : 
suppose that an investor is building a retirement portfolio
INPUT_TEXT : 
[CLS] Give me a real world example of volatility [SEP]   how does the vix work ? . in the world of investments , volatility is an indicator of how big ( or small ) moves a stock price , a sector - specific index , or a market - level index makes , and it represents how much risk is associated with the particular security , sector or market . the above stock - s

In [141]:
print("ANSWERS : ")
sortedAnswers = sorted(answerTuple, key = lambda x: x[1], reverse = True)

for item in sortedAnswers:
    print(item)

ANSWERS : 
('zomma', 4.393577)
('vix', 3.4838371)
('it is calculated in real - time based on the live prices of s & p 500 index', 2.586465)
('suppose that an investor is building a retirement portfolio', 1.710832)
('suppose that an investor is building a retirement portfolio', 1.710832)


In [552]:
## Timing Stemmer and Lemmatizer in both spacy and nltk ##

document = "Investors use a variety of tactics to ascertain risk. One of the most commonly used absolute risk metrics is standard deviation, a statistical measure of dispersion around a central tendency. You look at the average return of an investment and then find its average standard deviation over the same time period. Normal distributions (the familiar bell-shaped curve) dictate that the expected return of the investment is likely to be one standard deviation from the average 67% of the time and two standard deviations from the average deviation 95% of the time. This helps investors evaluate risk numerically. If they believe that they can tolerate the risk, financially and emotionally, they invest."

In [119]:
document = " Time      .    Hey"

In [203]:
## 1. Spacy Lemmatizer ##
nlp = spacy.load("en_core_web_sm",disable=['ner', 'parser', 'textcat'])

In [204]:
from time import time

start = time()
documentDoc = nlp(document)
nounLemma = [tokens.lemma_ for tokens in documentDoc if tokens.text.strip() != '']
end = time()
print(nounLemma)
print("TIME : ",end - start)

['Time', '.', 'hey']
TIME :  0.01604437828063965


In [205]:
len(topDocuments)

33

In [271]:
def convertQuery(query):
        
        nlp = spacy.load("en_core_web_sm",disable = ['ner', 'parser', 'textcat'])
        doc = nlp(query.lower())
        return doc

def getNouns(query):
        
        #nlp = spacy.load("en_core_web_sm")
        
        '''
        spacy_stopwords = list(spacy.lang.en.stop_words.STOP_WORDS)
        
        querySplit = self.query.lower().split()
        
        querySplit = [x for x in querySplit if x not in spacy_stopwords]
        
        print(querySplit)
        
        return querySplit
        
        '''
        nounList = []
        for token in query:
            #print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_, token.shape_, token.is_alpha, token.is_stop)
            if token.pos_ == "NOUN" or token.pos_ == "PROPN" or token.pos_ == "ADJ":
                nounList.append(token.text)
            
        #print("noun list : ", nounList)        
        return nounList
    
def checkNounMatch(paragraph,nounList):
        '''
        A function to see if each noun is matching for a paragraph
        '''
        nounBool = True
        
        #nlp = spacy.load("en_core_web_sm",disable=['ner', 'parser', 'textcat'])
        stemmer = SnowballStemmer("english")
        #paragraphContent = " ".join(paragraph)
        stemmedPara = [stemmer.stem(token) for token in paragraph]
        #paragraphDoc = nlp(paragraphContent)
        #paragraphLemma = [tokens.lemma_ for tokens in paragraphDoc] 
        #nounDoc = nlp(" ".join(nounList))
        #nounsLemma = [tokens.lemma_ for tokens in nounDoc]
        #paragraphLemma = [tokens.lemma_ for tokens in paragraphDoc if tokens.pos_ in ["NOUN","ADJ"]]
        for nouns in nounList: #nounsLemma:
            
            #nounLemma = [tokens.lemma_ for tokens in nounDoc]
            print(stemmer.stem(nouns))
            if stemmer.stem(nouns) in stemmedPara:#paragraphContent:
                nounBool = True
            else:
                nounBool = False
                break
        
        return nounBool
                
            
def returnParagraphList(content,nounList):
        '''
        Return pargaraph in the form of list of lists to be fed to the BM25 Model
        '''
        paragraphList = []
        sepCounter = 0
        #nounList = getNouns()
        
        for i,term in enumerate(content):
            
            if term == "--------------------------":
                paragraph = content[sepCounter:i] # " ".join() <-
                if checkNounMatch(paragraph,nounList):
                #if all(nouns in paragraph for nouns in nounList):
                    paragraphList.append(paragraph)
                
                sepCounter = i+1
        
        return paragraphList

In [272]:
len(topDocuments)

54

In [273]:
query = "What is implied volatility?"

In [262]:
queryList = convertQuery(query)

In [264]:
queryList

what are two common types of mbs?

In [265]:
nounList = getNouns(queryList)

In [266]:
nounList

['common', 'types', 'mbs']

In [267]:
for documents in topDocuments :
    
    
    start = time()
    paragraphList = returnParagraphList(documents[1],nounList)
    end = time()
    print("DOCUMENT : ",documents[0], " , TIME TAKEN : ",end - start)

DOCUMENT :  riskmanagement.asp.txt  , TIME TAKEN :  0.04189729690551758
DOCUMENT :  basispoint.asp.txt  , TIME TAKEN :  0.017946243286132812
DOCUMENT :  derivative.asp.txt  , TIME TAKEN :  0.061411142349243164
DOCUMENT :  equity.asp.txt  , TIME TAKEN :  0.051119089126586914
DOCUMENT :  mbs.asp.txt  , TIME TAKEN :  0.025313138961791992
DOCUMENT :  prospecttheory.asp.txt  , TIME TAKEN :  0.017072677612304688
DOCUMENT :  balancesheet.asp.txt  , TIME TAKEN :  0.03179740905761719
DOCUMENT :  bell-curve.asp.txt  , TIME TAKEN :  0.01796889305114746
DOCUMENT :  benchmarkerror.asp.txt  , TIME TAKEN :  0.009974241256713867
DOCUMENT :  calculating-covariance.asp.txt  , TIME TAKEN :  0.02587723731994629
DOCUMENT :  certificateofdeposit.asp.txt  , TIME TAKEN :  0.1207284927368164
DOCUMENT :  correlationcoefficient.asp.txt  , TIME TAKEN :  0.021047353744506836
DOCUMENT :  derivative.asp.txt  , TIME TAKEN :  0.06382894515991211
DOCUMENT :  equity.asp.txt  , TIME TAKEN :  0.05587577819824219
DOCUMENT 

In [236]:
stemmer = SnowballStemmer("english")

In [268]:
[stemmer.stem(x) for x in nounList]

['common', 'type', 'mbs']

In [275]:
document = "Types of Mortgage - Backed Securities . There are two common types of MBSs : pass - throughs and collateralized mortgage obligations ( CMO ) ."

In [276]:
document.split()

['Types',
 'of',
 'Mortgage',
 '-',
 'Backed',
 'Securities',
 '.',
 'There',
 'are',
 'two',
 'common',
 'types',
 'of',
 'MBSs',
 ':',
 'pass',
 '-',
 'throughs',
 'and',
 'collateralized',
 'mortgage',
 'obligations',
 '(',
 'CMO',
 ')',
 '.']

In [274]:
checkNounMatch(["There","are","two","common","types","of","mbss"],nounList)

common
type
mbs


False

In [367]:
stem = SnowballStemmer('english')

In [579]:
stem.stem('security')

'secur'

In [514]:
nlp = spacy.load("en_core_web_sm")

In [58]:
doc = nlp("What is implied volatility")

In [59]:
[tokens.pos_ for tokens in doc]

['PRON', 'AUX', 'VERB', 'NOUN']

In [585]:
x = "Securities there!!!"

In [586]:
doc = nlp(x)

In [587]:
query = [tokens.lemma_ for tokens in doc]

In [588]:
query

['security', 'there', '!', '!', '!']

TypeError: 'bool' object is not iterable

In [41]:
x = [1,2,3,4,4]
print([a for a in x while(a<3)]

SyntaxError: invalid syntax (<ipython-input-41-b2f087915276>, line 2)

In [7]:
"   dsf    fdf   ".replace(" ","")

'dsffdf'

In [45]:
os.getcwd()

'C:\\Users\\prateek.sethi\\Desktop\\BERT\\Website_QA\\updated_final_app'